In [1]:
# flake8: noqa
import warnings
import os

# Suppress noisy requests warnings.
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

# Scaling OCR using Ray Datasets

In this example, we will show you how to run optical character recognition (OCR) on a set of documents and analyze the resulting text with the natural language processing library [spaCy](https://spacy.io/). Running OCR on a large dataset is very computationally expensive, so using Ray for distributed processing can really speed up the processing. Ray Datasets makes it easy to compose the different steps of the pipeline, namely the OCR and the natural language processing.

To make it more interesting, we will run the analysis on the [LightShot](https://www.kaggle.com/datasets/datasnaek/lightshot) dataset. It is a large publicly available OCR dataset with a wide variety of different documents, all of them screen shots of various forms. It is easy to replace that dataset with your own data and adapt the example to your own use cases!

## Overview

This tutorial will cover:
 - Creating a Ray Dataset that represents the images in the dataset
 - Running the computationally expensive OCR process on each image in the dataset in parallel
 - Filtering out images that actually contain text
 - Performing various NLP operations on the text

## Walkthrough

Let's start by preparing the dependencies and downloading the dataset. First we install the OCR software `tesseract` and its Python client:

````{tabbed} macOS
```
brew install tesseract
pip install pytesseract
```
````

````{tabbed} linux
```
sudo apt-get install tesseract-ocr
pip install pytesseract
```
````

By default, the following example will run on a tiny dataset we provide. If you want to run it on the full dataset, we recommend to run it on a cluster since processing all the images with tesseract takes a lot of time.

````{note}
If you want to run the example on the full [LightShot](https://www.kaggle.com/datasets/datasnaek/lightshot) dataset, you need to download the dataset and extract it. You can extract the dataset by first running `unzip archive.zip` and then `unrar x LightShot13k.rar <target>`.
````

Let's now import Ray and initialize a local Ray cluster. If you want to run OCR at a very large scale, you should run this workload on a multi-node cluster.

In [2]:
# Import ray and initialize a local Ray cluster.
import ray
ray.init()

# Download the tiny version of the dataset.
import os
import urllib.request

tiny_dataset = [
    "https://upload.wikimedia.org/wikipedia/commons/2/27/QEMU_6.2_screenshot_%28cropped%29.png",
    "https://upload.wikimedia.org/wikipedia/commons/c/ce/GNOME_Screenshot_3.32.png",
    "https://upload.wikimedia.org/wikipedia/commons/f/f3/Miranda_IM_0.6.8_customized.png",
]

OCR_DATA = "/tmp/ocr_data/"

for i, url in enumerate(tiny_dataset):
    os.makedirs(OCR_DATA, exist_ok=True)
    urllib.request.urlretrieve(url, OCR_DATA + str(i) + ".png")

2022-06-21 18:14:02,451	INFO services.py:1476 -- View the Ray dashboard at http://127.0.0.1:8265


### Running the OCR software on the data

We first create a list `files` of absolute paths of the file names and then convert it into a Ray Dataset with the `ray.data.from_items` function. We can now run the `.map` function on this dataset of file names to run the actual OCR process on each file and convert the screen shots into text. Note that we only store the paths of the filenames and the OCR'ed text in the dataset to keep the size of the dataset manageble. If you have large binary blobs like images or videos, it can be beneficial to store them outside of the dataset and store only the extracted information in the dataset.

In [3]:
import os
from glob import glob
import pytesseract

files = glob(OCR_DATA + "*")
ds = ray.data.from_items(files)

def perform_ocr(path):
    return {"path": path, "text": pytesseract.image_to_string(path)}

results = ds.map(perform_ocr)

2022-06-21 18:14:12,151	WARNING dataset.py:3391 -- The `map`, `flat_map`, and `filter` operations are unvectorized and can be very slow. Consider using `.map_batches()` instead.
Map: 100%|██████████| 3/3 [00:01<00:00,  2.40it/s]


Let us have a look at some of the data points with the `take` function.

In [4]:
results.take(10)

[ArrowRow({'path': '/tmp/ocr_data/2.png',
           'text': '© Viktor (Online) : Message Session\n\n“etto| © Whter | steno\n\nremus\ntet? Fiviha\n\n17: dokonca to vie aj video @\nViktor\n\n1818. 55 samozrejme\n\nremus\n\n‘e138: hmm\n\ntere: a skusal si to uz?\n\nViktor\n\na:18: €€\n\n1818: len moj brat to skusal\nremus\n\nWA\n\nOG8hO38\n\nseettsgmailcom [0]\nonline\n\n@ Heenan a\n\nOnline\n\nsero\nBEE:..... 0088\n@ ore\n\naway\n\nTue\nhn\n\n& Wee\n\nYep, Tm here\n\n'}),
 ArrowRow({'path': '/tmp/ocr_data/1.png',
           'text': 'Take Screenshot\n\n© Grab the whole screen\nGrab the current window\n\n|_| eeeeeeter\n\nGrab after a delay of 0 +. seconds\n\nEffects\nInclude pointer\n¥ Include the window border\n\nApply effect: None Sa\n'}),
 ArrowRow({'path': '/tmp/ocr_data/0.png',
           'text': 'File Edit View Bookmarks\n\n[i New Tab [If] split view ~\n\n43044 kousekip\n\nPlugins\n\nkousekip:ako-kaede-mirai(htop)\n\nkousekip:ako-kaede-mirai(qemu-system-x86)\n\nSettings\n\nHelp\n\nk

### Process the extracted text data with spaCy

This is the part where the fun begins. Depending on your task there will be different needs for post processing, for example:
- If you are scanning books or articles you might want to separate the text out into sections and paragraphs.
- If you are scanning forms, receipts or checks, you might want to extract the different items listed, as well as extra information for those items like the price, or the total amount listed on the receipt or check.
- If you are scanning legal documents, you might want to extract information like the type of document, who is mentioned in the document and more semantic information about what the document claims.
- If you are scanning medical records, you might want to extract the patient name and the treatment history.

In our specific example, let's try to determine all the documents in the LightShot dataset that are chat protocols and extract named entities in those documents. We will extract this data with spaCy. Let's first make sure the libraries are installed:

In [ ]:
!pip install "spacy>=3"
!python -m spacy download en_core_web_sm
!pip install spacy_langdetect

This is some code to determine the language of a piece of text:

In [5]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

nlp = spacy.load('en_core_web_sm')

@Language.factory("language_detector")
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp.add_pipe('language_detector', last=True)
nlp("This is an English sentence. Ray rocks!")._.language

{'language': 'en', 'score': 0.9999974450049829}

It gives both the language and a confidence score for that language.

In order to run the code on the dataset, we should use Ray Datasets' built in support for actors since the `nlp` object is not serializable and we want to avoid having to recreate it for each individual sentence:

In [7]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

class SpacyBatchInference:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')

        @Language.factory("language_detector")
        def get_lang_detector(nlp, name):
           return LanguageDetector()

        self.nlp.add_pipe('language_detector', last=True)

    def __call__(self, row):
        row = dict(row)
        doc = self.nlp(row["text"])
        row["language"] = doc._.language["language"]
        row["score"] = doc._.language["score"]
        return row

results.limit(10).map(SpacyBatchInference, compute="actors")

Map Progress (1 actors 1 pending): 100%|██████████| 3/3 [00:07<00:00,  2.40s/it]


Dataset(num_blocks=3, num_rows=3, schema={path: string, text: string, language: string, score: double})

We can now get language statistics over the whole dataset:

In [8]:
languages = results.map(SpacyBatchInference, compute="actors")
languages.groupby("language").count().show()

Shuffle Reduce: 100%|██████████| 3/3 [00:00<00:00, 407.50it/s]

{'language': 'af', 'count()': 1}
{'language': 'en', 'count()': 2}


````{note}
On the full LightShot dataset, you would get the following:
```text
{'language': 'UNKNOWN', 'count()': 2815}
{'language': 'af', 'count()': 109}
{'language': 'ca', 'count()': 268}
{'language': 'cs', 'count()': 13}
{'language': 'cy', 'count()': 80}
{'language': 'da', 'count()': 33}
{'language': 'de', 'count()': 281}
{'language': 'en', 'count()': 5640}
{'language': 'es', 'count()': 453}
{'language': 'et', 'count()': 82}
{'language': 'fi', 'count()': 32}
{'language': 'fr', 'count()': 168}
{'language': 'hr', 'count()': 143}
{'language': 'hu', 'count()': 57}
{'language': 'id', 'count()': 128}
{'language': 'it', 'count()': 139}
{'language': 'lt', 'count()': 17}
{'language': 'lv', 'count()': 12}
{'language': 'nl', 'count()': 982}
{'language': 'no', 'count()': 56}
```
````

We can now filter to include only the English documents and also sort them according to their score.

In [11]:
languages.filter(lambda row: row["language"] == "en").sort("score", descending=True).take(1000)

Shuffle Reduce: 100%|██████████| 3/3 [00:00<00:00, 673.71it/s]


[ArrowRow({'path': '/tmp/ocr_data/1.png',
           'text': 'Take Screenshot\n\n© Grab the whole screen\nGrab the current window\n\n|_| eeeeeeter\n\nGrab after a delay of 0 +. seconds\n\nEffects\nInclude pointer\n¥ Include the window border\n\nApply effect: None Sa\n',
           'language': 'en',
           'score': 0.999997372590989}),
 ArrowRow({'path': '/tmp/ocr_data/2.png',
           'text': '© Viktor (Online) : Message Session\n\n“etto| © Whter | steno\n\nremus\ntet? Fiviha\n\n17: dokonca to vie aj video @\nViktor\n\n1818. 55 samozrejme\n\nremus\n\n‘e138: hmm\n\ntere: a skusal si to uz?\n\nViktor\n\na:18: €€\n\n1818: len moj brat to skusal\nremus\n\nWA\n\nOG8hO38\n\nseettsgmailcom [0]\nonline\n\n@ Heenan a\n\nOnline\n\nsero\nBEE:..... 0088\n@ ore\n\naway\n\nTue\nhn\n\n& Wee\n\nYep, Tm here\n\n',
           'language': 'en',
           'score': 0.7142816263267835})]

If you are interested in this example and want to extend it, you can do the following for the full dataset:
- go throught these results in order
- create labels on whether the text is a chat conversation and then train a model like [Huggingface Transformers](https://huggingface.co/docs/transformers/) on the data.

Contributions that extend the example in this direction with a PR are welcome!